In [4]:
from torch.utils.data import DataLoader, Dataset
import torch

class ContrastiveDataset(Dataset):
    def __init__(self, n=10000, num_groups=10):
        self.n = n
        self.num_groups = num_groups
        self.range_size = self.n // self.num_groups
        self.ranges = [
            (self.range_size * i, self.range_size * (i + 1)) for i in range(self.num_groups)
        ]

        # Handle any remaining samples by adding them to the last range
        if self.range_size * self.num_groups < self.n:
            self.ranges[-1] = (self.ranges[-1][0], self.n)

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        anchor = idx

        range_idx = anchor // self.range_size
        if range_idx >= self.num_groups:
            range_idx = self.num_groups - 1  # Correct adjustment

        start, end = self.ranges[range_idx]
        positive_sample = torch.randint(start, end, (1,)).item()

        return (anchor, positive_sample)
    
class Encoder(torch.nn.Module):
    def __init__(self, n):
        super(Encoder, self).__init__()
        self.embedding=torch.nn.Embedding(n, 768)
        self.fc=torch.nn.Linear(768, 768)
        self.non_linearity=torch.nn.Tanh()

    def forward(self, x):
        emb=self.embedding(x)
        out=self.fc(emb)
        out=self.non_linearity(out)
        return out
    
encoder=Encoder(10000)

# Initialize the dataset
dataset = ContrastiveDataset()

# Initialize DataLoader
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

# Iterate over DataLoader
for batch_idx, batch in enumerate(dataloader):
    #print(f"Batch {batch_idx + 1}: {batch}")
    print(encoder(batch[0]).shape)


torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size([10, 768])
torch.Size